Analysing the IROS Poster Data, for
object      : cylinder
object pose : 0,0,0
controller  : 1,2,3
with and without interaction

1. Analyse Control Data
2. Visualise Behaviour Data
3. Preprocess control & behaviour data
4. Analyse Data with MC_W, MC_MI, and MC_CW


### Functions & Variables

In [ ]:
parent = "/Users/zahedi/projects/TU.Berlin/experiments/poster";
name(hand,suffix,controller) = "$parent/rbohand$hand$suffix/hand0_0-controller$controller-objectcylinder_0.0_0.0_0.0"
shortname(hand,suffix,controller) = "rbohand$hand$suffix/hand0_0-controller$controller-objectcylinder_0.0_0.0_0.0"

function get_xyz(filename)
    fd = open(filename)
    lines = readlines(fd)
    close(fd)
    lines = lines[2:2:end]
    s = split(strip(lines[1]), " ")
    v = collect(1:7:length(s)-1)
    v = [[i,i+1,i+2] for i in v]
    indices = v[1]
    for i = 2:length(v)
        indices = vcat(indices, v[i])
    end
    indices = indices .+ 1
    values = zeros(length(lines), length(indices))
    index = 1
    for l in lines
        s = split(strip(l), " ")
        values[index,:] = [parse(Float64,v) for v in s[indices]]
        index = index + 1
    end
    return values
end

function get_xyz(hand, suffix, controller)
    filename = "$(name(hand,suffix,controller))/raw/hand.sofastates.txt"
    return get_xyz(filename)
end

check_differences(a, b, column) = sqrt(sum((a[column] - b[column]).^2))

In [ ]:
function plotHand(data,line,xmin,xmax,ymin,ymax,zmin,zmax)
    f = figure()
    xlim([xmin,xmax])
    ylim([ymin,ymax])
    zlim([zmin,zmax])
    plot3D(
    [data[line,1+3],data[line,1+2*3]],
    [data[line,2+3],data[line,2+2*3]],
    [data[line,3+3],data[line,3+2*3]]
    )

    for i in collect(1:5)
        plot3D(
        [data[line,1+i*3],data[line,1+(i+1)*3]],
        [data[line,2+i*3],data[line,2+(i+1)*3]],
        [data[line,3+i*3],data[line,3+(i+1)*3]])
    end
    # -> 6 is fingertip 1
    
    xlim([xmin,xmax])
    ylim([ymin,ymax])
    zlim([zmin,zmax])
    plot3D(
    [data[line,1+3],data[line,1+7*3]],
    [data[line,2+3],data[line,2+7*3]],
    [data[line,3+3],data[line,3+7*3]]
    )

    for i in collect(7:10)
        plot3D(
        [data[line,1+i*3],data[line,1+(i+1)*3]],
        [data[line,2+i*3],data[line,2+(i+1)*3]],
        [data[line,3+i*3],data[line,3+(i+1)*3]])
    end
    # -> 11 is fingertip 2

    plot3D(
    [data[line,1+3],data[line,1+12*3]],
    [data[line,2+3],data[line,2+12*3]],
    [data[line,3+3],data[line,3+12*3]]
    )

    for i in collect(12:15)
        plot3D(
        [data[line,1+i*3],data[line,1+(i+1)*3]],
        [data[line,2+i*3],data[line,2+(i+1)*3]],
        [data[line,3+i*3],data[line,3+(i+1)*3]])
    end
    # -> 16 is fingertip 3

    plot3D(
    [data[line,1+3],data[line,1+17*3]],
    [data[line,2+3],data[line,2+17*3]],
    [data[line,3+3],data[line,3+17*3]]
    )

    for i in collect(17:20)
        plot3D(
        [data[line,1+i*3],data[line,1+(i+1)*3]],
        [data[line,2+i*3],data[line,2+(i+1)*3]],
        [data[line,3+i*3],data[line,3+(i+1)*3]])
    end
    # -> 21 is fingertip 4

    plot3D(
    [data[line,1+3],data[line,1+22*3]],
    [data[line,2+3],data[line,2+22*3]],
    [data[line,3+3],data[line,3+22*3]]
    )

    for i in collect(22:30)
        plot3D(
        [data[line,1+i*3],data[line,1+(i+1)*3]],
        [data[line,2+i*3],data[line,2+(i+1)*3]],
        [data[line,3+i*3],data[line,3+(i+1)*3]])
    end
    savefig("/Users/zahedi/Desktop/plot_$line.png")
    close(f)
end

In [ ]:
function create_video(hand,suffix,controller)
    data = get_xyz(hand,suffix,controller)

    xmin = minimum([minimum(data[:,i]) for i in collect(1:3:size(data)[2])])
    xmax = maximum([maximum(data[:,i]) for i in collect(1:3:size(data)[2])])

    ymin = minimum([minimum(data[:,i]) for i in collect(2:3:size(data)[2])])
    ymax = maximum([maximum(data[:,i]) for i in collect(2:3:size(data)[2])])

    zmin = minimum([minimum(data[:,i]) for i in collect(3:3:size(data)[2])])
    zmax = maximum([maximum(data[:,i]) for i in collect(3:3:size(data)[2])])

    for i in 1:size(data)[1]
        plotHand(data,i,xmin,xmax,ymin,ymax,zmin,zmax)
    end
    
    name = shortname(hand,suffix,controller)
    name = replace(name,"/","_")

    run(`ffmpeg -y -framerate 5 -i /Users/zahedi/Desktop/plot_%d.png -c:v libx264 -r 30 -pix_fmt yuv420p /Users/zahedi/Desktop/$name.mp4`)

    for i in 1:size(data)[1]
        rm("/Users/zahedi/Desktop/plot_$i.png")
    end
end

# 1. Analyse Control Data

In [ ]:
using PyPlot
using DataFrames
using ProgressMeter

## Reading all control data

In [ ]:
rbohand2_controller_0_control_states = readtable("$(name(2,"",0))/raw/control.states.csv")
rbohand2_prescriptive_controller_0_control_states = readtable("$(name(2,"-prescriptive",0))/raw/control.states.csv")
rbohand3_controller_0_control_states = readtable("$(name(3,"",0))/raw/control.states.csv")
rbohand3_prescriptive_controller_0_control_states = readtable("$(name(3,"-prescriptive",0))/raw/control.states.csv")

rbohand2_controller_1_control_states = readtable("$(name(2,"",1))/raw/control.states.csv")
rbohand2_prescriptive_controller_1_control_states = readtable("$(name(2,"-prescriptive",1))/raw/control.states.csv")
rbohand3_controller_1_control_states = readtable("$(name(3,"",1))/raw/control.states.csv")
rbohand3_prescriptive_controller_1_control_states = readtable("$(name(3,"-prescriptive",1))/raw/control.states.csv")

rbohand2_controller_2_control_states = readtable("$(name(2,"",2))/raw/control.states.csv")
rbohand2_prescriptive_controller_2_control_states = readtable("$(name(2,"-prescriptive",2))/raw/control.states.csv")
rbohand3_controller_2_control_states = readtable("$(name(3,"",2))/raw/control.states.csv")
rbohand3_prescriptive_controller_2_control_states = readtable("$(name(3,"-prescriptive",2))/raw/control.states.csv")

In [ ]:
names1  = []
names2  = []
columns = []
l2      = []

for controller in [0,1,2]
    for hand1 in [2,3]
        for hand2 in [2,3]
            for column in [:wrist_x, :wrist_y, :wrist_z, :wrist_rx, :wrist_ry, :wrist_rz, :wrist_rw, :pressure_0, :pressure_1, :pressure_2, :pressure_3, :pressure_4, :pressure_5]
                name1 = name(hand1, "", controller)
                name2 = name(hand2, "-prescriptive", controller)
                if name1 != name2
                    a = readtable("$(name(hand1,"",controller))/raw/control.states.csv")
                    b = readtable("$(name(hand2,"-prescriptive",controller))/raw/control.states.csv")
                    append!(names1,  [shortname(hand1, "", controller)])
                    append!(names2,  [shortname(hand2, "-prescriptive", controller)])
                    append!(columns, [column])
                    append!(l2,[check_differences(a,b,column)])
                end
            end
        end
    end
end

for controller1 in [0,1,2]
    for controller2 in [0,1,2]
        for hand in [2,3]
            for column in [:wrist_x, :wrist_y, :wrist_z, :wrist_rx, :wrist_ry, :wrist_rz, :wrist_rw, :pressure_0, :pressure_1, :pressure_2, :pressure_3, :pressure_4, :pressure_5]
                name1 = name(hand, "", controller1)
                name2 = name(hand, "", controller2)
                if name1 != name2
                    a = readtable("$(name(hand,"",controller1))/raw/control.states.csv")
                    b = readtable("$(name(hand,"",controller2))/raw/control.states.csv")
                    append!(names1,  [shortname(hand, "", controller1)])
                    append!(names2,  [shortname(hand, "", controller2)])
                    append!(columns, [column])
                    append!(l2,[check_differences(a,b,column)])
                end
            end
        end
    end
end

differences_table = DataFrame()
differences_table[:First]  = names1
differences_table[:Second] = names2
differences_table[:Column] = columns
differences_table[:L2Norm] = l2
writetable("/Users/zahedi/Desktop/comparison.csv", differences_table)
differences_table

# 2. Analyse Behaviour Data

In [ ]:
rbohand2_prescriptive_controller_0_control_states = readtable("$(name(2,"-prescriptive",0))/raw/control.states.csv")

In [ ]:
for hand in [2,3]
    for suffix in ["", "-prescriptive"]
        for controller in [0,1,2]
            create_video(hand, suffix, controller)
        end
    end
end

# 3. Preprocess control and behaviour data

In [ ]:
parent = "/Users/zahedi/projects/TU.Berlin/experiments/poster";
subdir = readdir(parent)
subdir = filter(c->contains(c,"rbohand"),subdir)

dirs = []

for dir in subdir
    d = readdir("$parent/$dir")
    d = filter(c->contains(c,"hand"), d)
    d = ["$parent/$dir/$v" for v in d]
    append!(dirs, d)
end

dirs[1:10]

### 1) Preprocessing the files

In [ ]:
function copy_and_convert_hand(dirs)
    in_dirs  = map(x->"$x/raw", dirs)
    out_dirs = map(x->"$x/analysis", dirs)
    
    p = Progress(length(in_dirs)*1, 1, "hand.sofastates.csv ...")

    for index in 1:length(in_dirs)
        if isfile("$(in_dirs[index])/hand.sofastates.txt")
            m = get_xyz("$(in_dirs[index])/hand.sofastates.txt")
            for i = 5:3:size(m)[2]
                m[:,i] = m[:,i] - m[:,2]
            end
            m = m[:,4:end]
            if isdir(out_dirs[index]) == false
                mkdir(out_dirs[index])
            end
            writecsv("$(out_dirs[index])/hand.sofastates.csv",m)
        end
        next!(p)
    end
end

In [ ]:
function copy_and_convert_control(dirs)
    
    in_dirs  = map(x->"$x/raw", dirs)
    out_dirs = map(x->"$x/analysis", dirs)
    
    p = Progress(length(in_dirs), 1, "control.states.csv ...")

    for i in 1:length(in_dirs)
        if isfile("$(in_dirs[i])/control.states.csv")
            a = readcsv("$(in_dirs[i])/control.states.csv")[2:end-1,:]
            a = map(x->Float64(x),a)
            writecsv("$(out_dirs[i])/control.states.csv",a)
        end
        next!(p)
    end
end

function copy_and_convert_files(path)
    copy_and_convert_hand(path)
    copy_and_convert_control(path)
end

In [ ]:
copy_and_convert_files(dirs)

### 2)  Get Min/Max Values

In [ ]:
function write_domains(dirs)    
    in_dirs      = map(x->"$x/analysis", dirs)
    controlfiles = map(x->"$x/control.states.csv",in_dirs)
    a_domains    = map(x->"$x/A.domains.csv",in_dirs)
    handfiles    = map(x->"$x/hand.sofastates.csv",in_dirs)
    w_domains    = map(x->"$x/W.domains.csv",in_dirs)

    function minmax(in_,out_)
        a = readcsv(in_)
        b = minimum(a,1)
        c = maximum(a,1)
        d = vcat(b,c)
        writecsv(out_, d)
    end

    p = Progress(length(in_dirs), 1, "domain files ...")

    for i = 1:length(in_dirs)
        if isfile(controlfiles[i])
            minmax(controlfiles[i], a_domains[i])
        end
        if isfile(handfiles[i])
            minmax(handfiles[i],w_domains[i])
        end
        next!(p)
    end
end

write_domains(dirs)

In [ ]:
w_indices = join(["$(i*3+1),$(i*3+2),$(i*3+3)" for i in [5,10,15,20]],",")

### 3)  Hand, Controller, Objects, Positions

In [ ]:
wbins     = 100
abins     = 30

# index 1 was removed when copying (wrist position)
# -> 5 is fingertip 1
# -> 10 is fingertip 2
# -> 15 is fingertip 3
# -> 20 is fingertip 4

w_indices = join(["$(i*3+1),$(i*3+2),$(i*3+3)" for i in [5,10,15,20]],",")
a_indices = "8,9,10,11"

in_dirs = map(x->"$x/analysis", dirs)

fd = open("Makefile.1","w")
write(fd,"all:\n")
for d in in_dirs
    command = "/Users/zahedi/projects/builds/entropy-ninja/bin/rbo_mc -wi $(w_indices) -ai $(a_indices) -wbins $wbins -abins $abins -d \"$d\""   
    write(fd,"\t$command\n")
end
close(fd)
run(`make -j 4 -f Makefile.1`)

### SKIPPED 4) Hand, Controller, Objects, Average over positions

In [ ]:
parents = map(x->replace(x, r"^(.+object\w+)_.+$", s"\g<1>"),dirs)
parents = unique(parents)

for d in parents
    mkpath("$path/combined/object_position/$d/analysis")
end

pg = Progress(length(parents), 1, "combine hand.sofastates.csv and control.states.csv ...")

for p in parents
    ds = filter(x->contains(x,"$(p)_"),dirs)
    hand = nothing
    control = nothing
    
    for a in ds
        pp = "$path/$a/analysis/hand.sofastates.csv"
        if isfile(pp)
            h = readcsv(pp)
            if hand == nothing
                hand = h
            else
                hand = vcat(hand,h)
            end
        else
            println(">  missing $pp")
        end

        pp = "$path/$a/analysis/control.states.csv"
        if isfile(pp)
            c = readcsv(pp)
            if control == nothing
                control = c
            else
                control = vcat(control,c)
            end
        else
            println(">  missing $pp")
        end
    end
    if hand != nothing
        writecsv("$path/combined/object_position/$p/analysis/hand.sofastates.csv",hand)
    end
    if control != nothing
        writecsv("$path/combined/object_position/$p/analysis/control.states.csv", control)
    end
    next!(pg)
end

In [ ]:
write_domains("$path/combined/object_position/")

In [ ]:
parents = map(x->replace(x, r"^(.+object\w+)_.+$", s"\g<1>"),dirs)
parents = unique(parents)

fd = open("Makefile.2","w")
write(fd,"all:\n")
for p in parents
    command = "/Users/zahedi/projects/builds/entropy-build/bin/rbo_mc -wi $(w_indices) -ai $(a_indices) -wbins $wbins -abins $abins -d \"$path/combined/object_position/$p/analysis\""   
    write(fd,"\t$command\n")
end
close(fd)
run(`make -j 4 -f Makefile.2`)

### 5) Hand, Controller, Avg over Objects, Position = 0.0, 0.0, 0.0

In [ ]:
dirs = readdir(path)
dirs = filter(x->contains(x,"hand"), dirs)
dirs = filter(x->contains(x,"_0.0_0.0_0.0"), dirs)

parents = map(x->replace(x, r"^(.+)-object\w+_0.0_0.0_0.0", s"\g<1>"),dirs)
parents = unique(parents)

for d in parents
    mkpath("$path/combined/object_position_0/$d/analysis")
end

for p in parents
    println(p)
    ds = filter(x->contains(x,"$(p)"),dirs)
    hand = nothing
    control = nothing
    
    for a in ds
        pp = "$path/$a/analysis/hand.sofastates.csv"
        #println(pp)
        #println(a)
        if isfile(pp)
            h = readcsv(pp)
            if hand == nothing
                hand = h
            else
                hand = vcat(hand,h)
            end
        else
            println(">  missing $pp")
        end

        pp = "$path/$a/analysis/control.states.csv"
        if isfile(pp)
            c = readcsv(pp)
            if control == nothing
                control = c
            else
                control = vcat(control,c)
            end
        else
            println(">  missing $pp")
        end
    end
    if hand != nothing
        writecsv("$path/combined/object_position_0/$p/analysis/hand.sofastates.csv",hand)
    end
    if control != nothing
        writecsv("$path/combined/object_position_0/$p/analysis/control.states.csv", control)
    end
    next!(pg)
end

write_domains("$path/combined/object_position_0/")

dirs = readdir(path)
dirs = filter(x->contains(x,"hand"), dirs)
dirs = filter(x->contains(x,"_0.0_0.0_0.0"), dirs)

parents = map(x->replace(x, r"^(.+)-object\w+_0.0_0.0_0.0", s"\g<1>"),dirs)
parents = unique(parents)

fd = open("Makefile.3","w")
write(fd,"all:\n")
for p in parents
    command = "/Users/zahedi/projects/builds/entropy-build/bin/rbo_mc -wi $(w_indices) -ai $(a_indices) -wbins $wbins -abins $abins -d \"$path/combined/object_position_0/$p/analysis\""   
    write(fd,"\t$command\n")
end
close(fd)
run(`make -j 4 -f Makefile.3`)

### 6) Hand, Controller, Avg over Objects, Avg Position

In [ ]:
dirs = readdir(path)
dirs = filter(x->contains(x,"hand"), dirs)

parents = map(x->replace(x, r"^(.+)-object.+", s"\g<1>"),dirs)
parents = unique(parents)

for d in parents
    mkpath("$path/combined/hand_control/$d/analysis")
end

for p in parents
    println(p)
    ds = filter(x->contains(x,"$(p)"),dirs)
    hand = nothing
    control = nothing
    
    for a in ds
        pp = "$path/$a/analysis/hand.sofastates.csv"
        #println(pp)
        #println(a)
        if isfile(pp)
            h = readcsv(pp)
            if hand == nothing
                hand = h
            else
                hand = vcat(hand,h)
            end
        else
            println(">  missing $pp")
        end

        pp = "$path/$a/analysis/control.states.csv"
        if isfile(pp)
            c = readcsv(pp)
            if control == nothing
                control = c
            else
                control = vcat(control,c)
            end
        else
            println(">  missing $pp")
        end
    end
    if hand != nothing
        writecsv("$path/combined/hand_control/$p/analysis/hand.sofastates.csv",hand)
    end
    if control != nothing
        writecsv("$path/combined/hand_control/$p/analysis/control.states.csv", control)
    end
    next!(pg)
end

write_domains("$path/combined/hand_control/")

dirs = readdir(path)
dirs = filter(x->contains(x,"hand"), dirs)

parents = map(x->replace(x, r"^(.+)-object.+", s"\g<1>"),dirs)
parents = unique(parents)

fd = open("Makefile.4","w")
write(fd,"all:\n")
for p in parents
    command = "/Users/zahedi/projects/builds/entropy-build/bin/rbo_mc -wi $(w_indices) -ai $(a_indices) -wbins $wbins -abins $abins -d \"$path/combined/hand_control/$p/analysis\""   
    write(fd,"\t$command\n")
end
close(fd)
run(`make -j 4 -f Makefile.4`)

### Auswertung

In [ ]:
dirs = readdir("$path/combined/hand_control")
in_dirs = filter(x->contains(x,"hand"), dirs)

mc_files = map(x->"$path/combined/hand_control/$x/analysis/mc_w-averaged_$(wbins)_$(abins).txt", in_dirs)
values   = map(x->readcsv(x)[1],mc_files)
fd = open("$path/combined/hand_control/results_mc_w-averaged_$(wbins)_$(abins).txt","w")
for i = 1:length(in_dirs)
    write(fd, "$(in_dirs[i]),$(values[i])\n")
end
close(fd)

In [ ]:
dirs = readdir("$path/combined/object_position")
in_dirs = filter(x->contains(x,"hand"), dirs)

mc_files = map(x->"$path/combined/object_position/$x/analysis/mc_w-averaged_$(wbins)_$(abins).txt", in_dirs)
values   = map(x->readcsv(x)[1],mc_files)
fd = open("$path/combined/object_position/results_mc_w-averaged_$(wbins)_$(abins).txt","w")
for i = 1:length(in_dirs)
    write(fd, "$(in_dirs[i]),$(values[i])\n")
end
close(fd)

In [ ]:
dirs = readdir("$path/combined/object_position_0")
in_dirs = filter(x->contains(x,"hand"), dirs)

mc_files = map(x->"$path/combined/object_position_0/$x/analysis/mc_w-averaged_$(wbins)_$(abins).txt", in_dirs)
values   = map(x->readcsv(x)[1],mc_files)
fd = open("$path/combined/object_position_0/results_mc_w-averaged_$(wbins)_$(abins).txt","w")
for i = 1:length(in_dirs)
    write(fd, "$(in_dirs[i]),$(values[i])\n")
end
close(fd)

In [ ]:
dirs = readdir("$path")
in_dirs = filter(x->contains(x,"hand"), dirs)

mc_files = map(x->"$path/$x/analysis/mc_w-averaged_$(wbins)_$(abins).txt", in_dirs)
values   = map(x->readcsv(x)[1],mc_files)
fd = open("$path/results_mc_w-averaged_$(wbins)_$(abins).txt","w")
for i = 1:length(in_dirs)
    write(fd, "$(in_dirs[i]),$(values[i])\n")
end
close(fd)